In [34]:
np.random.seed(0)

In [35]:
import numpy as np

In [36]:
def leapfrog(x, v, gradient, timestep, trajectory_length):
    print("__________________")
    print("iter")
    print("x is :", x)
    print("v is :", v)
    print("*******")
    v -= 0.5 * timestep * gradient(x)
    for _ in range(trajectory_length - 1):
        print("x is :", x)
        print("v is :", v)
        print("Grad x", gradient(x))
        x += timestep * v
        v -= timestep * gradient(x)
    x += timestep * v
    v -= 0.5 * timestep * gradient(x)
    print("x is :", x)
    print("v is :", v)
    return x, v

In [45]:
def sample_HMC(x_old, log_prob, log_prob_gradient, timestep, trajectory_length):
    # switch to physics mode!
    def E(x): return -log_prob(x)
    def gradient(x): return -log_prob_gradient(x)
    def K(v): return 0.5 * np.sum(v ** 2)
    def H(x, v): return K(v) + E(x)

    # Metropolis acceptance probability, implemented in "logarithmic space"
    # for numerical stability:
    def log_p_acc(x_new, v_new, x_old, v_old):
        return min(0, -(H(x_new, v_new) - H(x_old, v_old)))

    # give a random kick to particle by drawing its momentum from p(v)
    v_old = np.array([0.5, 0.5])

    # approximately calculate position x_new and momentum v_new after
    # time trajectory_length  * timestep
    x_new, v_new = leapfrog(x_old.copy(), v_old.copy(), gradient, 
                            timestep, trajectory_length)

    # accept / reject based on Metropolis criterion
    accept = np.log(np.random.random()) < log_p_acc(x_new, v_new, x_old, v_old)
    print("MH:", log_p_acc(x_new, v_new, x_old, v_old))

    # we consider only the position x (meaning, we marginalize out v)
    if accept:
        return accept, x_new
    else:
        return accept, x_old

In [46]:
def build_HMC_chain(init, timestep, trajectory_length, n_total, log_prob, gradient):
    n_accepted = 0
    chain = [init]

    for _ in range(n_total):
        accept, state = sample_HMC(chain[-1].copy(), log_prob, gradient,
                                   timestep, trajectory_length)
        chain.append(state)
        n_accepted += accept

    acceptance_rate = n_accepted / float(n_total)

    return chain, acceptance_rate

In [47]:
def log_prob(x): return -0.5 * np.sum(x ** 2)

In [48]:
def log_prob_gradient(x): return -x

In [49]:
chain, acceptance_rate = build_HMC_chain(np.array([5.0, 1.0]), 1.5, 10, 100,
                                         log_prob, log_prob_gradient)
print("Acceptance rate: {:.3f}".format(acceptance_rate))

__________________
iter
x is : [5. 1.]
v is : [0.5 0.5]
*******
x is : [5. 1.]
v is : [-3.25 -0.25]
Grad x [5. 1.]
x is : [0.125 0.625]
v is : [-3.4375 -1.1875]
Grad x [0.125 0.625]
x is : [-5.03125 -1.15625]
v is : [4.109375 0.546875]
Grad x [-5.03125 -1.15625]
x is : [ 1.1328125 -0.3359375]
v is : [2.41015625 1.05078125]
Grad x [ 1.1328125 -0.3359375]
x is : [4.74804688 1.24023438]
v is : [-4.71191406 -0.80957031]
Grad x [4.74804688 1.24023438]
x is : [-2.31982422  0.02587891]
v is : [-1.23217773 -0.84838867]
Grad x [-2.31982422  0.02587891]
x is : [-4.16809082 -1.2467041 ]
v is : [5.0199585  1.02166748]
Grad x [-4.16809082 -1.2467041 ]
x is : [3.36184692 0.28579712]
v is : [-0.02281189  0.5929718 ]
Grad x [3.36184692 0.28579712]
x is : [3.32762909 1.17525482]
v is : [-5.01425552 -1.16991043]
Grad x [3.32762909 1.17525482]
x is : [-2.27919054 -1.03035212]
v is : [2.98576868 0.47226989]
MH: 0
__________________
iter
x is : [-2.27919054 -1.03035212]
v is : [0.5 0.5]
*******
x is : [-2.

x is : [-0.90407838 -0.90407838]
v is : [0.52494874 0.52494874]
Grad x [-0.90407838 -0.90407838]
x is : [-0.11665527 -0.11665527]
v is : [0.69993165 0.69993165]
Grad x [-0.11665527 -0.11665527]
x is : [0.9332422 0.9332422]
v is : [-0.69993165 -0.69993165]
Grad x [0.9332422 0.9332422]
x is : [-0.11665527 -0.11665527]
v is : [-0.52494874 -0.52494874]
Grad x [-0.11665527 -0.11665527]
x is : [-0.90407838 -0.90407838]
v is : [0.83116883 0.83116883]
Grad x [-0.90407838 -0.90407838]
x is : [0.34267487 0.34267487]
v is : [0.31715653 0.31715653]
Grad x [0.34267487 0.34267487]
x is : [-0.54727728 -0.54727728]
v is : [-0.5 -0.5]
MH: -2.2191393167503293e-10
__________________
iter
x is : [-0.54727728 -0.54727728]
v is : [0.5 0.5]
*******
x is : [-0.54727728 -0.54727728]
v is : [0.91045796 0.91045796]
Grad x [-0.54727728 -0.54727728]
x is : [0.81840966 0.81840966]
v is : [-0.31715653 -0.31715653]
Grad x [0.81840966 0.81840966]
x is : [0.34267487 0.34267487]
v is : [-0.83116883 -0.83116883]
Grad x [

Grad x [-0.11665527 -0.11665527]
x is : [0.9332422 0.9332422]
v is : [-0.69993165 -0.69993165]
Grad x [0.9332422 0.9332422]
x is : [-0.11665527 -0.11665527]
v is : [-0.52494874 -0.52494874]
Grad x [-0.11665527 -0.11665527]
x is : [-0.90407838 -0.90407838]
v is : [0.83116883 0.83116883]
Grad x [-0.90407838 -0.90407838]
x is : [0.34267487 0.34267487]
v is : [0.31715653 0.31715653]
Grad x [0.34267487 0.34267487]
x is : [-0.54727728 -0.54727728]
v is : [-0.5 -0.5]
MH: 0
__________________
iter
x is : [-0.54727728 -0.54727728]
v is : [0.5 0.5]
*******
x is : [-0.54727728 -0.54727728]
v is : [0.91045796 0.91045796]
Grad x [-0.54727728 -0.54727728]
x is : [0.81840966 0.81840966]
v is : [-0.31715653 -0.31715653]
Grad x [0.81840966 0.81840966]
x is : [0.34267487 0.34267487]
v is : [-0.83116883 -0.83116883]
Grad x [0.34267487 0.34267487]
x is : [-0.90407838 -0.90407838]
v is : [0.52494874 0.52494874]
Grad x [-0.90407838 -0.90407838]
x is : [-0.11665527 -0.11665527]
v is : [0.69993165 0.69993165]

x is : [-0.54727728 -0.54727728]
v is : [-0.5 -0.5]
MH: 0
__________________
iter
x is : [-0.54727728 -0.54727728]
v is : [0.5 0.5]
*******
x is : [-0.54727728 -0.54727728]
v is : [0.91045796 0.91045796]
Grad x [-0.54727728 -0.54727728]
x is : [0.81840966 0.81840966]
v is : [-0.31715653 -0.31715653]
Grad x [0.81840966 0.81840966]
x is : [0.34267487 0.34267487]
v is : [-0.83116883 -0.83116883]
Grad x [0.34267487 0.34267487]
x is : [-0.90407838 -0.90407838]
v is : [0.52494874 0.52494874]
Grad x [-0.90407838 -0.90407838]
x is : [-0.11665527 -0.11665527]
v is : [0.69993165 0.69993165]
Grad x [-0.11665527 -0.11665527]
x is : [0.9332422 0.9332422]
v is : [-0.69993165 -0.69993165]
Grad x [0.9332422 0.9332422]
x is : [-0.11665527 -0.11665527]
v is : [-0.52494874 -0.52494874]
Grad x [-0.11665527 -0.11665527]
x is : [-0.90407838 -0.90407838]
v is : [0.83116883 0.83116883]
Grad x [-0.90407838 -0.90407838]
x is : [0.34267487 0.34267487]
v is : [0.31715653 0.31715653]
Grad x [0.34267487 0.34267487]

Grad x [-0.54727728 -0.54727728]
x is : [0.81840966 0.81840966]
v is : [-0.31715653 -0.31715653]
Grad x [0.81840966 0.81840966]
x is : [0.34267487 0.34267487]
v is : [-0.83116883 -0.83116883]
Grad x [0.34267487 0.34267487]
x is : [-0.90407838 -0.90407838]
v is : [0.52494874 0.52494874]
Grad x [-0.90407838 -0.90407838]
x is : [-0.11665527 -0.11665527]
v is : [0.69993165 0.69993165]
Grad x [-0.11665527 -0.11665527]
x is : [0.9332422 0.9332422]
v is : [-0.69993165 -0.69993165]
Grad x [0.9332422 0.9332422]
x is : [-0.11665527 -0.11665527]
v is : [-0.52494874 -0.52494874]
Grad x [-0.11665527 -0.11665527]
x is : [-0.90407838 -0.90407838]
v is : [0.83116883 0.83116883]
Grad x [-0.90407838 -0.90407838]
x is : [0.34267487 0.34267487]
v is : [0.31715653 0.31715653]
Grad x [0.34267487 0.34267487]
x is : [-0.54727728 -0.54727728]
v is : [-0.5 -0.5]
MH: 0
__________________
iter
x is : [-0.54727728 -0.54727728]
v is : [0.5 0.5]
*******
x is : [-0.54727728 -0.54727728]
v is : [0.91045796 0.91045796]

x is : [-0.90407838 -0.90407838]
v is : [0.83116883 0.83116883]
Grad x [-0.90407838 -0.90407838]
x is : [0.34267487 0.34267487]
v is : [0.31715653 0.31715653]
Grad x [0.34267487 0.34267487]
x is : [-0.54727728 -0.54727728]
v is : [-0.5 -0.5]
MH: 0
__________________
iter
x is : [-0.54727728 -0.54727728]
v is : [0.5 0.5]
*******
x is : [-0.54727728 -0.54727728]
v is : [0.91045796 0.91045796]
Grad x [-0.54727728 -0.54727728]
x is : [0.81840966 0.81840966]
v is : [-0.31715653 -0.31715653]
Grad x [0.81840966 0.81840966]
x is : [0.34267487 0.34267487]
v is : [-0.83116883 -0.83116883]
Grad x [0.34267487 0.34267487]
x is : [-0.90407838 -0.90407838]
v is : [0.52494874 0.52494874]
Grad x [-0.90407838 -0.90407838]
x is : [-0.11665527 -0.11665527]
v is : [0.69993165 0.69993165]
Grad x [-0.11665527 -0.11665527]
x is : [0.9332422 0.9332422]
v is : [-0.69993165 -0.69993165]
Grad x [0.9332422 0.9332422]
x is : [-0.11665527 -0.11665527]
v is : [-0.52494874 -0.52494874]
Grad x [-0.11665527 -0.11665527]

In [44]:
np.random.random()

0.3494402920485349